In [10]:
import pandas as pd

import os
from sklearn.preprocessing import MinMaxScaler
from statsmodels.formula.api import ols

import pandas as pd 
import numpy as np
import statsmodels.api as sm


csv_list=[]
file =os.listdir()
for i in file:
    if ( i[-3:]=='csv'):
        csv_list.append(i)


In [11]:
def make_ols_model(csv_list):
    for i in csv_list:
        df = pd.read_csv(i,encoding='euc-kr')      
        #회귀분석을 위한 표준화 , 변수 선택
        scaler = MinMaxScaler()        
        df_minmax_scaled = scaler.fit_transform(df.iloc[:,[8,9,10,11,12,13,14]])
        df_minmax_scaled = pd.DataFrame(df_minmax_scaled,columns=df.iloc[:,[8,9,10,11,12,13,14]].columns)
        cob = (df[['폐업수']].max()-df[['폐업수']]) / ( df[['폐업수']].max()-df[['폐업수']].min() )
        data_set = pd.concat([df_minmax_scaled,cob],axis=1)

        x_data = data_set.iloc[:,[0,1,2,3,5,7]] #변수 여러개
        target = data_set.iloc[:,6]

        # for b0, 상수항 추가
        x_data1 = sm.add_constant(x_data, has_constant = "add")

        # OLS 검정
        multi_model = sm.OLS(target, x_data1)
        fitted_multi_model = multi_model.fit()
        fitted_multi_model.summary()

        data_reg = pd.DataFrame([list(fitted_multi_model.params[1:])],columns=[fitted_multi_model.params.index[1:]])
        data_set=data_set.iloc[:,[0,1,2,3,5,7,6]]
        #가중치
        data_reg=data_reg* (fitted_multi_model.rsquared_adj)
        for j in range(len(data_reg.columns)):
            data_set.iloc[:,j] = data_set.iloc[:,j] * abs(data_reg.iloc[0,j])
        data_set.iloc[:,-1] = data_set.iloc[:,-1] * (1-fitted_multi_model.rsquared_adj)
        data_set['score'+i[7:13]] = data_set.sum(axis=1)
        #최종데이터셋
        data = pd.concat([df.iloc[:,:6],data_set],axis=1)
        print(i[7:13]+"_adj R_squared 값",fitted_multi_model.rsquared_adj)
        #print("###가중치###",data_reg)
        data.to_csv('회귀분석/OLS_'+i,encoding='euc-kr',index=False)
    
    

In [12]:
make_ols_model(csv_list)

201712_adj R_squared 값 0.7028993365460848
201806_adj R_squared 값 0.7240000498449375
201812_adj R_squared 값 0.6948801541869087
201906_adj R_squared 값 0.6544596883641147
201912_adj R_squared 값 0.6151137818409822
202006_adj R_squared 값 0.5321541731382629
202012_adj R_squared 값 0.41315448171764046
202106_adj R_squared 값 0.3558903689431612
202112_adj R_squared 값 0.3423956952044156
202206_adj R_squared 값 -0.001385430506276597


In [323]:
fa_list = os.listdir('회귀분석/')

sum_fa_list = pd.DataFrame()
sum_fa_list= pd.read_csv('회귀분석/'+fa_list[0],encoding='euc-kr').iloc[:,:6]
for i in fa_list:
    df=pd.read_csv('회귀분석/'+i,encoding='euc-kr')
    sum_fa_list = pd.concat([sum_fa_list,df[df.columns[-1]]],axis=1)
    
sum_fa_list.to_csv('회귀분석/sum_ols_list.csv',encoding='euc-kr',index=False)

# 라인별 실행

In [13]:
df = pd.read_csv(csv_list[-1],encoding='euc-kr')

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

df_minmax_scaled = scaler.fit_transform(df.iloc[:,[8,9,10,11,12,13,14]])
df_minmax_scaled = pd.DataFrame(df_minmax_scaled,columns=df.iloc[:,[8,9,10,11,12,13,14]].columns)
cob = (df[['폐업수']].max()-df[['폐업수']]) / ( df[['폐업수']].max()-df[['폐업수']].min() )
data_set = pd.concat([df_minmax_scaled,cob],axis=1)

In [17]:
df

,상권명,다중지역정보,업종정보,위도,경도,wkt,상권코드,price_202206_max,strc_202206_max,pop_202206_max,상가수,area,유동인구 평균,매출금액 평균,매출액_시점 개수,폐업수,폐업수_시점 개수
0,성남은행초등학교,"{""type"":""Polygon"",""coordinates"":[[[127.1687262...",두발 미용업/일반 교과학원/치킨 전문점/한식 일반 음식점업/노래 연습장 운영업/기타...,37.460705,127.168758,"POLYGON ((127.1687262760 37.4620601200, 127.16...",골목_1,41449260.29,304,8826,138,74330.36627,523283.95000,2.301757e+09,6,6,6
1,가림로201번안길,"{""type"":""Polygon"",""coordinates"":[[[126.8617425...",전자상거래 소매업/기타 음ㆍ식료품 위주 종합 소매업/자동차 세차업/두발 미용업,37.475776,126.861436,"POLYGON ((126.8617425380 37.4748298650, 126.86...",골목_2,28678797.34,425,5761,11,44415.19650,134826.49330,8.888940e+07,6,0,6
2,내각1로73번안길,"{""type"":""Polygon"",""coordinates"":[[[127.1649527...",한식 일반 음식점업/두발 미용업/부동산 중개 및 대리업/한식 육류 요리 전문점/노래...,37.711797,127.161510,"POLYGON ((127.1649527610 37.7043120880, 127.16...",골목_3,17764535.32,344,4166,94,425896.39240,155463.77670,4.875092e+08,6,7,6
3,양주산성로,"{""type"":""Polygon"",""coordinates"":[[[126.9873577...",두발 미용업/일반 교과학원/화초 및 식물 소매업/통신기기 소매업/한식 육류 요리 전...,37.788623,126.987220,"POLYGON ((126.9873577160 37.7908375740, 126.98...",골목_4,10425021.67,41,479,27,162099.07560,32086.21333,2.372946e+08,6,3,6
4,도일로135번길,"{""type"":""Polygon"",""coordinates"":[[[126.7835814...",두발 미용업/치킨 전문점/체인화 편의점/한식 일반 음식점업/기타 음ㆍ식료품 위주 종...,37.343110,126.783057,"POLYGON ((126.7835814060 37.3430218760, 126.78...",골목_5,13240664.49,153,7150,22,41766.72043,171490.26330,3.448667e+08,6,2,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1672,광명사거리역_4번출구,"{""type"":""Polygon"",""coordinates"":[[[126.8536079...",기타 주점업/한식 일반 음식점업/한식 육류 요리 전문점/노래 연습장 운영업/커피 전...,37.479619,126.852135,"POLYGON ((126.8536079430 37.4782792810, 126.85...",발달_853,41496885.44,274,7189,150,28388.93889,318564.44500,4.697315e+09,6,10,6
1673,성복2로,"{""type"":""Polygon"",""coordinates"":[[[127.0773354...",일반의원/두발 미용업/한식 일반 음식점업/의약품 및 의료용품 소매업/기타 주점업/셔...,37.313360,127.078773,"POLYGON ((127.0773354300 37.3137514240, 127.07...",발달_854,18347782.84,21,2348,81,14307.02069,50591.96333,3.652328e+09,6,4,6
1674,광교중앙로,"{""type"":""Polygon"",""coordinates"":[[[127.0687753...",두발 미용업/일반 교과학원/피자 햄버거 샌드위치 및 유사 음식점업/한식 육류 요...,37.296904,127.068295,"POLYGON ((127.0687753130 37.2965215900, 127.06...",발달_855,31741731.17,20,7343,166,36489.58697,159336.34330,4.740135e+09,6,17,6
1675,서수원버스터미널,"{""type"":""Polygon"",""coordinates"":[[[126.9719643...",기타 의복 소매업/화장품 비누 및 방향제 소매업/체인화 편의점/신발 소매업/속옷 ...,37.282524,126.970408,"POLYGON ((126.9719643130 37.2822193820, 126.97...",발달_856,29777090.07,76,1727,53,41186.95950,358778.70000,1.030756e+10,6,1,6


In [14]:
from statsmodels.formula.api import ols

import pandas as pd 
import numpy as np
import statsmodels.api as sm


#다중 선형회귀분석
x_data = data_set.iloc[:,[0,1,2,3,5,7]] #변수 여러개
target = data_set.iloc[:,6]

# for b0, 상수항 추가
x_data1 = sm.add_constant(x_data, has_constant = "add")

# OLS 검정
multi_model = sm.OLS(target, x_data1)
fitted_multi_model = multi_model.fit()
fitted_multi_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              매출액_시점 개수   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.6135
Date:                Thu, 24 Nov 2022   Prob (F-statistic):              0.720
Time:                        15:11:15   Log-Likelihood:                 2812.9
No. Observations:                1677   AIC:                            -5612.
Df Residuals:                    1670   BIC:                            -5574.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               1.0077      0.023     43.536      0.000       0.962       1.053
strc_202206_max     0.0100      0.013      0.764      0.445      -0.016       0.036
pop_202206_max      0.0028      0.011      0.261      0.794      -0.018       0.024
상가수                -0.0010      0.027     -0.039      0.969      -0.054       0.052
area                0.0039      0.022      0.174      0.862      -0.040       0.048
매출금액 평균             0.0062      0.045      0.136      0.892      -0.083       0.095
폐업수                -0.0126      0.023     -0.544      0.587      -0.058       0.033
==============================================================================
Omnibus:                     3804.679   Durbin-Watson:                   2.009
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         13968284.216
Skew:                         -20.978   Prob(JB):                         0.00
Kurtosis:                     448.133   Cond. No.                         60.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [310]:
#소수점 10자리까지나타나게
pd.options.display.float_format = '{:10f}'.format
#데이터셋 확인
fitted_multi_model.params.index[1:]

Index(['price_202006_max', 'strc_202006_max', 'pop_202006_max', '상가수',
       '유동인구 평균', '폐업수'],
      dtype='object')

In [311]:
data_reg = pd.DataFrame([list(fitted_multi_model.params[1:])],columns=[fitted_multi_model.params.index[1:]])
data_set=data_set.iloc[:,[0,1,2,3,5,7,6]]
data_reg=data_reg*fitted_multi_model.rsquared
for i in range(len(data_reg.columns)):
    data_set.iloc[:,i] = data_set.iloc[:,i] * abs(data_reg.iloc[0,i])

In [312]:
#매출액 가중치
data_set.iloc[:,-1] = data_set.iloc[:,-1] * (1-fitted_multi_model.rsquared)

In [313]:
#모델설명력
fitted_multi_model.rsquared

0.5338290388668849

In [314]:
data_set['score'] = data_set.sum(axis=1)

In [315]:
data = pd.concat([df.iloc[:,:6],data_set],axis=1)